# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [13]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt

In [ ]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 38
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

In [ ]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

In [16]:
#df.to_csv("data/BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [ ]:
#print(df)
indices_to_drop = df[df['reviews'].str.contains('Not Verified')].index

# Drop those rows
data = df.drop(indices_to_drop)





In [ ]:
data['reviews'] = data['reviews'].apply(lambda x: x[17:])

data.to_csv("data/BA_reviews.csv")

In [ ]:
chunked_data = []
good = 0
bad = 0
total = 0
x = 0
for chunk in pd.read_csv("data/BA_reviews.csv", chunksize=1):
    # Process each row (chunk) here
    for index, row in chunk.iterrows():
        analysis = TextBlob(str(row[1]))
        sentiment = analysis.sentiment.polarity
        if sentiment > 0:
            good = good + 1
        if sentiment < 0:
            bad = bad + 1

        x=x+1
        
        total = total + sentiment
        chunked_data.append(sentiment)
        #chunked_data.append(row[1])
    average_sentiment = total/x



In [ ]:
data['sentiment score'] = chunked_data

data.to_csv("data/BA_reviews.csv")

In [ ]:
print(good,bad,average_sentiment)

categories = ['Good', 'Bad']
values = [good, bad]
plt.bar(categories, values)
plt.xlabel('Categories')
plt.ylabel('Values')
plt.title('No. good and bad reviews')


In [ ]:
import pandas as pd
import numpy as np
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Load the data
df = pd.read_csv('data\BA_reviews.csv')

# Preprocess the text data
def preprocess(text):
    return simple_preprocess(text, deacc=True)  # Tokenize and remove punctuation

df['processed_reviews'] = df['reviews'].apply(preprocess)

# Create a dictionary representation of the documents
dictionary = Dictionary(df['processed_reviews'])

# Filter out extremes to limit the number of features
dictionary.filter_extremes(no_below=15, no_above=0.5)

# Create a corpus: list of bag-of-words
corpus = [dictionary.doc2bow(doc) for doc in df['processed_reviews']]

# Set parameters
num_topics = 5  # Number of topics

# Build LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

# Prepare the visualization
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization
pyLDAvis.display(vis_data)


In [ ]:
from wordcloud import WordCloud
text = " ".join(review for review in df.reviews)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()